# **Classifieur n°1 : pertinence globale**



## **Importation**

In [1]:
#!pip install -U spacy
#!python -m spacy download fr_core_news_md

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/text_mining/Notebooks


In [5]:
import pandas as pd
import spacy
from collections import defaultdict, Counter
import numpy as np
from sklearn.externals import joblib

#Importation pré-traitement
import nltk
from nltk.tokenize.regexp import WordPunctTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import unicodedata
nlp = spacy.load('fr_core_news_md')

#Strop words
def strip_accents(texte):
  return(unicodedata.normalize('NFKD', texte).encode('ASCII', 'ignore').decode('ASCII'))
nltk.download('stopwords')
sw=stopwords.words("french")
sw += ['être','avoir']
sw= [strip_accents(w) for w in sw]

#Importation modèles
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

#Importation données
QA = pd.read_csv('../Data/Q&A.csv',sep=";")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Pré-traitement**

In [0]:
#Définition des fonctions de prétraitement du texte
def lemmatise_text(text):
  tw_nlp = nlp(text)
  list_lem = [token.lemma_ for token in tw_nlp]
  text_lem = ' '.join(list_lem)
  return text_lem

def stem_text(text):
  tokenizer = WordPunctTokenizer()
  stemmer = SnowballStemmer('french')
  liste_racines = [stemmer.stem(token) for token in tokenizer.tokenize(text)]
  return ' '.join(liste_racines)

def normalise(text):
  #stop words, strip accent et lowercase vont être fait automatiquement
  text = text.replace('\n','').replace('\r','').split(" ")
  text = " ".join([i for i in text if i!=""])
  lemmas = lemmatise_text(text) #lemme de notre texte
  stems = stem_text(lemmas) #stem de notre texte A VOIR
  return stems

In [0]:
#Une partie du corpus conversationnel
f = open("../Data/OpenSubtitles.fr-is.fr", mode='r', encoding='utf-8')
corpus_conv = f.readlines()[:202]
themes_corpus_conv = ['autres']*len(corpus_conv)

#Rassemblement questions et réponses pour créer un corpus (= liste de documents)
corpus_QA = list(QA.Answers) + list(QA.Questions)
themes_corpus_QA = ['metier']*len(corpus_QA)

#Création du corpus pour le classifieur de pertinence globale
corpus1 = corpus_conv + corpus_QA
themes1 = themes_corpus_conv + themes_corpus_QA

#Normalisation du corpus
collection1 = pd.Series(corpus1)
collection1 = collection1.apply(normalise)

In [0]:
#Création train et validation
X_train, X_valid, y_train, y_valid = train_test_split(collection1,
                                                      themes1,
                                                      train_size=0.75,
                                                      random_state=5)

In [0]:
#Création du vectorizer bin sur X_train : BINAIRE
vectorizer = CountVectorizer(lowercase=True,
                             stop_words=sw,
                             strip_accents='unicode',
                             binary=True).fit(X_train)
X_train_vectorized_bin = vectorizer.transform(X_train)
X_valid_vectorized_bin = vectorizer.transform(X_valid)

In [0]:
#Création du vectorizer count sur X_train : COUNT
vectorizer = CountVectorizer(lowercase=True,
                             stop_words=sw,
                             strip_accents='unicode',
                             binary=False).fit(X_train)
X_train_vectorized_count = vectorizer.transform(X_train)
X_valid_vectorized_count = vectorizer.transform(X_valid)

In [0]:
#Création du vectorizer tfidf sur X_train : TFIDF
vectorizer = TfidfVectorizer(lowercase=True,
                             stop_words=sw,
                             strip_accents='unicode',
                             max_df=0.5).fit(X_train)
X_train_vectorized_tfidf = vectorizer.transform(X_train)
X_valid_vectorized_tfidf = vectorizer.transform(X_valid)

In [0]:
resultat_binaire = {}
resultat_count = {}
resultat_tfidf = {}

## **SVM**
**Vectorizer Binaire**

In [0]:
param_grid = {'C': np.arange(0.1,10,0.1),'gamma':np.arange(0.009,1,0.1)} 
mvs = SVC(kernel = 'rbf')
grid = model_selection.GridSearchCV(estimator=mvs,param_grid=param_grid,cv = 4, scoring='accuracy', n_jobs = -1)

In [15]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_bin, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_bin)
predictions_train = grille.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['SVM']=accuracy_score(y_valid, predictions_valid)

{'C': 4.6, 'gamma': 0.109} 0.9075438596491228
1.0 0.8613861386138614


**Vectorizer count**

In [16]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_count, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_count)
predictions_train = grille.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['SVM']=accuracy_score(y_valid, predictions_valid)

{'C': 4.6, 'gamma': 0.109} 0.9108333333333334
1.0 0.8613861386138614


**Vectorizer tfidf**

In [17]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_tfidf, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_tfidf)
predictions_train = grille.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['SVM']=accuracy_score(y_valid, predictions_valid)

{'C': 5.0, 'gamma': 0.109} 0.9207894736842105
0.9966996699669967 0.8712871287128713


## **Régression logistique**
**Vectorizer Binaire**

In [0]:
param_grid = {'C': np.arange(0.5,5,0.1)} 
mlr = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter = 1500)
grid = model_selection.GridSearchCV(estimator=mlr,param_grid=param_grid,cv = 4, scoring='accuracy', n_jobs = -1)

In [19]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_bin, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_bin)
predictions_train = grille.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['log']=accuracy_score(y_valid, predictions_valid)

{'C': 0.5} 0.8943421052631579
0.9702970297029703 0.8217821782178217


**Vectorizer count**

In [20]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_count, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_count)
predictions_train = grille.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['log']=accuracy_score(y_valid, predictions_valid)

{'C': 4.199999999999999} 0.900921052631579
1.0 0.8415841584158416


**Vectorizer tfidf**

In [21]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_tfidf, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_tfidf)
predictions_train = grille.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['log']=accuracy_score(y_valid, predictions_valid)

{'C': 1.6999999999999997} 0.9141666666666667
1.0 0.8811881188118812


## **Random Forest**
**Vectorizer Binaire**

In [0]:
model_rf = RandomForestClassifier(n_estimators=1000, oob_score = True)

In [23]:
model_rf.fit(X_train_vectorized_bin, y_train)
predictions_valid = model_rf.predict(X_valid_vectorized_bin)
predictions_train = model_rf.predict(X_train_vectorized_bin)
print(accuracy_score(y_train,predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['rf']=accuracy_score(y_valid, predictions_valid)

1.0 0.8118811881188119


**Vectorizer count**

In [24]:
model_rf.fit(X_train_vectorized_count, y_train)
predictions_valid = model_rf.predict(X_valid_vectorized_count)
predictions_train = model_rf.predict(X_train_vectorized_count)
print(accuracy_score(y_train,predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['rf']=accuracy_score(y_valid, predictions_valid)

1.0 0.7920792079207921


**Vectorizer tfidf**

In [25]:
model_rf.fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_rf.predict(X_valid_vectorized_tfidf)
predictions_train = model_rf.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train,predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['rf']=accuracy_score(y_valid, predictions_valid)

1.0 0.7920792079207921


## **Ridge Regression**
**Vectorizer Binaire**

In [0]:
param_grid = {'alpha': [0.001,0.01,0.01,0.1,1,10,100]}
mridge = RidgeClassifier(tol=0.01)
grid = model_selection.GridSearchCV(estimator=mridge,param_grid=param_grid,cv = 4, scoring='accuracy', n_jobs = -1)

In [27]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_bin, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_bin)
predictions_train = grille.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['ridge']=accuracy_score(y_valid, predictions_valid)

{'alpha': 10} 0.8514035087719298
0.9405940594059405 0.7821782178217822


**Vectorizer count**

In [28]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_count, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_count)
predictions_train = grille.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['ridge']=accuracy_score(y_valid, predictions_valid)

{'alpha': 1} 0.8282456140350877
1.0 0.7326732673267327


**Vectorizer tfidf**

In [29]:
#Recherche des meilleurs paramètres
grille = grid.fit(X_train_vectorized_tfidf, y_train)
print(grille.best_params_,grille.best_score_)
predictions_valid = grille.predict(X_valid_vectorized_tfidf)
predictions_train = grille.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['ridge']=accuracy_score(y_valid, predictions_valid)

{'alpha': 1} 0.9141666666666667
1.0 0.8712871287128713


## **Naif bayésien**
**Vectorizer binaire**

In [30]:
model_nb = MultinomialNB().fit(X_train_vectorized_bin, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_bin)
predictions_train = model_nb.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['naive']=accuracy_score(y_valid, predictions_valid)

0.9603960396039604 0.8811881188118812


**Vectorizer count**

In [31]:
model_nb = MultinomialNB().fit(X_train_vectorized_count, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_count)
predictions_train = model_nb.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['naive']=accuracy_score(y_valid, predictions_valid)

0.9471947194719472 0.8811881188118812


**Vectorizer tfidf**

In [32]:
model_nb = MultinomialNB().fit(X_train_vectorized_tfidf, y_train)
predictions_valid = model_nb.predict(X_valid_vectorized_tfidf)
predictions_train = model_nb.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['naive']=accuracy_score(y_valid, predictions_valid)

0.9867986798679867 0.9108910891089109


## **Boosting**
**Vectorizer binaire**

In [0]:
param_grid = {'learning_rate':[0.01,0.1,1.0,10.0]}
mboosting = GradientBoostingClassifier()
grid = model_selection.GridSearchCV(estimator=mboosting,param_grid=param_grid,cv = 4, scoring='accuracy', n_jobs = -1)

In [34]:
grille = grid.fit(X_train_vectorized_bin, y_train)
predictions_valid = grille.predict(X_valid_vectorized_bin)
predictions_train = grille.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['boosting']=accuracy_score(y_valid, predictions_valid)

1.0 0.7821782178217822


**Vectorizer count**

In [35]:
grille = grid.fit(X_train_vectorized_count, y_train)
predictions_valid = grille.predict(X_valid_vectorized_count)
predictions_train = grille.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['boosting']=accuracy_score(y_valid, predictions_valid)

0.9702970297029703 0.8415841584158416


**Vectorizer tfidf**

In [36]:
grille = grid.fit(X_train_vectorized_tfidf, y_train)
predictions_valid = grille.predict(X_valid_vectorized_tfidf)
predictions_train = grille.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['boosting']=accuracy_score(y_valid, predictions_valid)

1.0 0.7722772277227723


## **AdaBoostClassifier**
**Vectorizer binaire**

In [0]:
param_grid = {'learning_rate':[0.01,0.1,1.0,10.0], 'n_estimators':[50,60,70,80,90,100]}
madaboost = AdaBoostClassifier()
grid = model_selection.GridSearchCV(estimator=madaboost,param_grid=param_grid,cv = 4, scoring='accuracy', n_jobs = -1)

In [38]:
grille = grid.fit(X_train_vectorized_bin, y_train)
predictions_valid = grille.predict(X_valid_vectorized_bin)
predictions_train = grille.predict(X_train_vectorized_bin)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_binaire['adaboost']=accuracy_score(y_valid, predictions_valid)

0.9867986798679867 0.8712871287128713


**Vectorizer count**

In [39]:
grille = grid.fit(X_train_vectorized_count, y_train)
predictions_valid = grille.predict(X_valid_vectorized_count)
predictions_train = grille.predict(X_train_vectorized_count)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_count['adaboost']=accuracy_score(y_valid, predictions_valid)

0.9867986798679867 0.8712871287128713


**Vectorizer tfidf**

In [40]:
grille = grid.fit(X_train_vectorized_tfidf, y_train)
predictions_valid = grille.predict(X_valid_vectorized_tfidf)
predictions_train = grille.predict(X_train_vectorized_tfidf)
print(accuracy_score(y_train, predictions_train),accuracy_score(y_valid, predictions_valid))
resultat_tfidf['adaboost']=accuracy_score(y_valid, predictions_valid)

1.0 0.8316831683168316


## **Restitution de l'ensemble des résultats**

In [41]:
resultat = [resultat_binaire,resultat_count,resultat_tfidf]
pd.DataFrame(resultat,index=['binaire','count','tfidf'])

,SVM,log,rf,ridge,naive,boosting,adaboost
binaire,0.861386,0.821782,0.811881,0.782178,0.881188,0.782178,0.871287
count,0.861386,0.841584,0.792079,0.732673,0.881188,0.841584,0.871287
tfidf,0.871287,0.881188,0.792079,0.871287,0.910891,0.772277,0.831683


## **Apprentissage du meilleur modèle sur l'ensemble du jeu de données et enregistrement en pickle**

In [0]:
vectorizer1 = TfidfVectorizer(lowercase=True,
                             stop_words=sw,
                             strip_accents='unicode',
                             max_df=0.5).fit(collection1)
collection_vectorized_tfidf = vectorizer1.transform(collection1)
classifieur1 = MultinomialNB().fit(collection_vectorized_tfidf, themes1)

In [43]:
joblib.dump(classifieur1, '../Data/classifieur1.pkl')
joblib.dump(vectorizer1, '../Data/vectoriseur1.pkl')

['../Data/vectoriseur1.pkl']